In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="bwISTK3XdVI7LVTIYM62")
project = rf.workspace("zwdyolov5").project("seg_real")
dataset = project.version(1).download("png-mask-semantic")


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to seg_real-1 in png-mask-semantic:: 100%|██████████| 366/366 [00:00<00:00, 7261.63it/s]


In [1]:
import time
import copy
import torch
from torch import optim, nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.models import resnet18
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import sys
sys.path.append("..")
from IPython import display
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore") # 忽略警告

In [2]:
import numpy as np

In [3]:
import numpy as np
from pathlib import Path
import os
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


def transfer_label(label):
    width, height, channel = label.shape
    res = np.zeros_like(label)
    channel = label[:, :, 0]
    for row_idx in range(height):
        for col_idx in range(width):
            color = channel[row_idx, col_idx]
            channel_map = {0 : 0, 1 : 1, 2 : 2}
            res[row_idx, col_idx, channel_map[channel[row_idx, col_idx]]] = 1
    return res

def transfer_label_2class(label):
    width, height, _ = label.shape
    channel = label[:, :, 0]
    res = np.zeros((640, 640, 1))
    
    for row_idx in range(height):
        for col_idx in range(width):
            if channel[row_idx, col_idx] == 1:
                res[row_idx, col_idx, 0] = 1 
                
    return res
            
def load_image(type):
    root = f"./seg_real-1/{type}"
    mask_image_list = []
    origin_image_list = []
    
    for root, dirs, files in os.walk(root):
        for file in files:
            mask_file_path = os.path.join(root, file)
            if mask_file_path[-3:] == "png":
                mask_frame = cv2.imread(mask_file_path)
                mask_frame_rgb = mask_frame[:, :, ::-1].copy()
                mask_frame_label = transfer_label_2class(mask_frame_rgb)
                mask_image_list.append(mask_frame_label)
                
                origin_file_path = mask_file_path[:-9] + ".jpg"
                origin_frame = cv2.imread(origin_file_path)
                origin_frame_rgb = origin_frame[:, :, ::-1].copy()
                origin_image_list.append(origin_frame_rgb)
                
    mask_image_np = np.vstack(mask_image_list).reshape((-1, 640, 640, 1))
    origin_image_np = np.vstack(origin_image_list).reshape((-1, 640, 640, 3))
    
    images, labels = shuffle(origin_image_np, mask_image_np)
    print(f'images.shape: {images.shape}')
    print(f'labels.shape: {labels.shape}')
    return images, labels

In [86]:
train_images, train_labels = load_image('train')
test_images, test_labels = load_image('test')
val_images, val_labels = load_image('valid')

(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 640, 1)
(640, 

## Model

In [4]:
# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [99]:
def create_model(input_shape, pool_size):
    # 创建网络模型
    model = Sequential()
    # 对输入层进行归一化处理
    model.add(BatchNormalization(input_shape=input_shape))

    # 卷积层1，名为Conv1
    model.add(Conv2D(4, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'))

    # 卷积层2
    model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))

    # 最大化层
    model.add(MaxPooling2D(pool_size=pool_size))

    # 卷积层3
    model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
    model.add(Dropout(0.2))

    # 卷积层4
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
    model.add(Dropout(0.2))

    # 卷积层5
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
    model.add(Dropout(0.2))

    # 最大化层2
    model.add(MaxPooling2D(pool_size=pool_size))
    
    # 上采样层 2
    model.add(UpSampling2D(size=pool_size))

    # 反卷积层 3
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'))
    model.add(Dropout(0.2))

    # 反卷积层 4
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'))
    model.add(Dropout(0.2))

    # 反卷积层 5
    model.add(Conv2DTranspose(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'))
    model.add(Dropout(0.2))

    # 上采样层 3
    model.add(UpSampling2D(size=pool_size))

    # 反卷积层 6
    model.add(Conv2DTranspose(4, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'))

    # 输出层
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'sigmoid', name = 'Final'))

    return model

## 训练模型

In [100]:
pool_size = (2, 2)
input_shape = train_images.shape[1:]
model = create_model(input_shape, pool_size)

# datagen = ImageDataGenerator(channel_shift_range=0.2)
# datagen.fit(images)

# 编译模型
model.compile(optimizer='Adam', loss='mean_squared_error')
# 可视化模型
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_6 (Bat  (None, 640, 640, 3)       12        
 chNormalization)                                                
                                                                 
 Conv1 (Conv2D)              (None, 638, 638, 4)       112       
                                                                 
 Conv2 (Conv2D)              (None, 636, 636, 8)       296       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 318, 318, 8)       0         
 ng2D)                                                           
                                                                 
 Conv3 (Conv2D)              (None, 316, 316, 8)       584       
                                                                 
 dropout_60 (Dropout)        (None, 316, 316, 8)      

In [107]:
epochs = 20
model.fit(train_images, train_labels, batch_size=30, epochs=epochs, validation_data=(val_images, val_labels))

Epoch 1/20
6/6 [==============================] - 30s 5s/step - loss: 0.0575 - val_loss: 0.0481
Epoch 2/20
6/6 [==============================] - 27s 4s/step - loss: 0.0333 - val_loss: 0.0517
Epoch 3/20
6/6 [==============================] - 29s 5s/step - loss: 0.0273 - val_loss: 0.0398
Epoch 4/20
6/6 [==============================] - 27s 4s/step - loss: 0.0250 - val_loss: 0.0454
Epoch 5/20
6/6 [==============================] - 27s 4s/step - loss: 0.0242 - val_loss: 0.0532
Epoch 6/20
6/6 [==============================] - 27s 4s/step - loss: 0.0236 - val_loss: 0.0327
Epoch 7/20
6/6 [==============================] - 26s 4s/step - loss: 0.0227 - val_loss: 0.0349
Epoch 8/20
6/6 [==============================] - 28s 5s/step - loss: 0.0224 - val_loss: 0.0271
Epoch 9/20
6/6 [==============================] - 27s 4s/step - loss: 0.0221 - val_loss: 0.0248
Epoch 10/20
6/6 [==============================] - 27s 4s/step - loss: 0.0216 - val_loss: 0.0255
Epoch 11/20
6/6 [======================

In [108]:
model.save('latest_try2.h5')

## 使用模型

In [5]:
from IPython.display import clear_output, Image, display, HTML

def create_video(path, model):
    vs = cv2.VideoCapture(path)
    while True:
        (grabbed, frame_source) = vs.read()
        if not grabbed: break
        
        height, width = frame_source.shape[:2]
        
        frame = cv2.resize(frame_source ,(640, 640))
        
        # 为frame_input添加一个维度
        frame_input = frame[None, :, :, :]
        prediction = model.predict(frame_input)
        prediction = prediction[0] * 255
        
        blank = cv2.resize(prediction, (width, height))
        output = cv2.addWeighted(frame_source, 0.3, blank, 0.7, 0, dtype = cv2.CV_32F)
        print(output)
        
        # 清空绘图空间
        clear_output(wait=True)
        
        # 显示处理结果
        _, jpg = cv2.imencode('.jpg', output)
        display(jpg)

        #按键盘中的q键退出检测
        if cv2.waitKey(1) & 0xFF == ord('q'):   break
        
    # 释放资源
    print("[INFO] cleaning up...")
    vs.release()
    cv2.destroyAllWindows()


In [6]:
from keras.models import load_model
def deal_with_video(video_path, model_path):
    save_dir = "./seg_video/"
    # if os.path.exists(save_dir):
    #     import shutil
    #     shutil.rmtree(save_dir, ignore_errors=True)
    # os.mkdir(save_dir)
    save_path = f"{save_dir}/seg_video.mp4"
    
    source = cv2.VideoCapture(video_path)
    
    fps = source.get(cv2.CAP_PROP_FPS)
    w = int(source.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(source.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    model = load_model(model_path)
    
    while True:
        ret, frame = source.read()
        if ret == False:    break
        
        frame_resize = cv2.resize(frame, (640, 640))
        prediction = model.predict(frame_resize[None, :, :, :])[0]
        
        blank = np.zeros((640, 640, 3))
        blank[:, :, 1] = prediction[:, :, 0]
        blank = blank * 255
        
        blank = cv2.resize(blank, (w, h))
        new_frame = cv2.addWeighted(frame, 0.3, blank, 0.7, 0, dtype = cv2.CV_8U)
        output = cv2.resize(new_frame, (w, h))
                
        vid_writer.write(output)
    
    # for idx, (frame, frame_rgb) in enumerate(source):
    #     vid_writer.write(frame)
        
deal_with_video('../pose/test.mp4', './latest_try2.h5')

NameError: name 'model' is not defined

In [ ]:
from keras.models import load_model
import cv2
model = load_model('second_try.h5')

frame_source = cv2.imread('./road-segmenttation-6/test/IMG_6089_jpg.rf.7bc033675cb0a80f24f8a91aae978a64.jpg')
height, width = frame_source.shape[:2]
frame = cv2.resize(frame_source ,(640, 640))
# 为frame_input添加一个维度
frame_input = frame[None, :, :, :]
prediction = model.predict(frame_input)
prediction = prediction[0] * 255


blank = cv2.resize(prediction, (width, height))
output = cv2.addWeighted(frame_source, 0, blank, 1, 0, dtype = cv2.CV_32F)

cv2.imwrite('test.png', output)


In [73]:
import os
import cv2
from keras.models import load_model
model = load_model('first_try.h5')
for root, dirs, files in os.walk('./seg_real-1/train'):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path[-3:] == "jpg":
            frame_source = cv2.imread(file_path)
            height, width = frame_source.shape[:2]
            frame = cv2.resize(frame_source ,(640, 640))
            # 为frame_input添加一个维度
            frame_input = frame[None, :, :, :]
            prediction = model.predict(frame_input)
            prediction = prediction[0] * 255

            blank = cv2.resize(prediction, (width, height))
            output = cv2.addWeighted(frame_source, 0.3, blank, 0.7, 0, dtype = cv2.CV_32F)
            cv2.imwrite(f'./generate/{file}', output)


1/1 [==============================] - 0s 164ms/step


In [ ]:
import cv2
from keras.models import load_model
import numpy as np

frame = cv2.imread('./road-segmenttation-6/train/1_jpg.rf.1f002cde2f7e14e6d12e6d8ea7fbfe5f.jpg')
model = load_model('third_try.h5')

frame = cv2.resize(frame ,(640, 640))
# 为frame_input添加一个维度
frame_input = frame[None, :, :, :]
            
predict = model.predict(frame_input)

flatten = predict.reshape(-1)
set(flatten.tolist())

In [44]:
label = cv2.imread('./seg_real-1/train/1686399388294716_mp4-0_jpg.rf.367e373d2e7f550d020f112b5bb4f96a_mask.png')
test = label[:,:,0].reshape(-1)
set(test.tolist())

{0, 1, 2}

In [43]:
test = train_labels[:, :, :, :].reshape(-1)
set(test.tolist())

{0, 1}

## 验证

In [103]:
def show_blank(image , label):
    blank = np.zeros((640, 640, 3))
    blank[:, :, 0] = label[:, :, 0]
    blank = blank * 255
    cv2.imwrite('./generate/1.jpg', blank)
    
    cv2.imwrite('./generate/1ori.jpg', image)

label = train_labels[0]
image = train_images[0]
show_blank(image, label)

In [110]:
model = load_model('latest_try2.h5')
prediction = model.predict(train_images)

6/6 [==============================] - 8s 1s/step


In [111]:
label = prediction[0]
image = train_images[0]
show_blank(image, label)